In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import yfinance as yf
import plotly.graph_objects as go
import plotly.figure_factory as ff
from scipy.stats import norm

# U.S. STOCK MARKET RETURN

## VOLATILITY

In [17]:
mkt = pdr('F-F_Research_Data_Factors_Daily','famafrench',start=1926)[0]/100
mkt = mkt['Mkt-RF'] + mkt.RF
mkt_vol = mkt.resample("M").apply(lambda x: x.std())[:-1]
mkt_vol.index = mkt_vol.index.to_period("M").astype(str)
mkt_vol.name = "market volatility"
mkt_vol.tail()


Date
2022-11    0.018173
2022-12    0.011684
2023-01    0.010742
2023-02    0.010208
2023-03    0.011875
Name: market volatility, dtype: float64

## MONTHLY RETURNS

In [18]:
mkt = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[0]/100
mkt_ret = mkt['Mkt-RF'] + mkt.RF
mkt_ret.index = mkt_ret.index.astype(str)
mkt_ret.name = "market return"
mkt_ret.tail()

Date
2022-12   -0.0608
2023-01    0.0700
2023-02   -0.0224
2023-03    0.0287
2023-04    0.0096
Name: market return, dtype: float64

# RETURNS FROM YAHOO FINANCE

In [13]:
ticker = input("Input a ticker")
price = yf.download(ticker, start="1970-01-01")["Adj Close"]

[*********************100%***********************]  1 of 1 completed


## VOLATILITY

In [15]:
ret_daily = price.pct_change()
ret_vol = ret_daily.resample("M").apply(lambda x: x.std())[:-1]
ret_vol.index = ret_vol.index.to_period("M").astype(str)
ret_vol.name = ticker + " volatility"
ret_vol.tail()

C:\Users\kerry\AppData\Local\Temp\ipykernel_55436\3651085163.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  ret_vol.index = ret_vol.index.to_period("M").astype(str)


Date
2023-01    0.016254
2023-02    0.016534
2023-03    0.013760
2023-04    0.012627
2023-05    0.012628
Name: aapl volatility, dtype: float64

## MONTHLY RETURNS

In [19]:
price_monthly = price.resample("M").last()
ret_ret = price_monthly.pct_change()[:-1]
ret_ret.index = ret_ret.index.to_period("M").astype(str)
ret_ret.index.name = "month"
ret_ret.name = ticker + " return"
ret_ret.tail()

C:\Users\kerry\AppData\Local\Temp\ipykernel_55436\774543617.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  ret_ret.index = ret_ret.index.to_period("M").astype(str)


month
2023-01    0.110521
2023-02    0.023183
2023-03    0.118649
2023-04    0.028987
2023-05    0.046058
Name: aapl return, dtype: float64

# SELECT YAHOO RETURN OR MARKET

In [20]:
flag = input("input 0 for market or 1 for yahoo return")
flag = "mkt" if flag=="0" else "ret"
name = "Market" if flag=="mkt" else ticker.upper()
vol = eval(flag + "_vol")
ret = eval(flag + "_ret")

In [ ]:
trace = go.Scatter(x=m.month, y=m.vol, mode="lines", hovertemplate="%{x}<br>%{y:.1%}<extra></extra>")
    fig1 = go.Figure(trace)
    fig1.update_yaxes(tickformat=".0%")
    fig1.layout.yaxis["title"] = f"{tick} Volatility"

    fig = px.scatter(
        m,
        x="vol_lag",
        y="vol",
        trendline="ols",
        hover_data=dict(ret=False, vol=False, month=False, vol_lag=False),
        hover_name="month",
    )
    fig.layout.xaxis["title"] = f"Lagged {tick} Volatility"
    fig.layout.yaxis["title"] = f"{tick} Volatility"
    fig.update_traces(
        marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
        selector=dict(mode="markers"),
    )
    fig.update_yaxes(tickformat=".0%")
    fig.update_xaxes(tickformat=".0%")
    fig2 = fig

    fig = px.scatter(
        m,
        x="vol_lag",
        y="ret",
        trendline="ols",
        hover_data=dict(ret=False, vol=False, month=False, vol_lag=False),
        hover_name="month",
    )
    fig.layout.xaxis["title"] = f"Lagged {tick} Volatility"
    fig.layout.yaxis["title"] = f"Monthly {tick} Return"
    fig.update_traces(
        marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
        selector=dict(mode="markers"),
    )
    fig.update_yaxes(tickformat=".0%")
    fig.update_xaxes(tickformat=".0%")
    fig3 = fig

    fig = px.scatter(
        m,
        x="vol",
        y="ret",
        trendline="ols",
        hover_data=dict(ret=False, vol=False, month=False, vol_lag=False),
        hover_name="month",
    )
    fig.layout.xaxis["title"] = f"{tick} Volatility"
    fig.layout.yaxis["title"] = f"Monthly {tick} Return"
    fig.update_traces(
        marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
        selector=dict(mode="markers"),
    )
    fig.update_yaxes(tickformat=".0%")
    fig.update_xaxes(tickformat=".0%")

# FIGURE 1

In [ ]:
trace1 = go.Box(
    y=daily, 
    text=daily.index.to_list(), 
    name=name + " Daily", 
    hovertemplate="%{text}<br>%{y:.1%}<extra></extra>"
)
trace2 = go.Box(
    y=monthly, 
    text=monthly.index.to_list(), 
    name=name + " Monthly", 
    hovertemplate="%{text}<br>%{y:.1%}<extra></extra>"
)
trace3 = go.Box(
    y=annual, 
    text=annual.index.to_list(), 
    name=name + " Annual", 
    hovertemplate="%{text}<br>%{y:.1%}<extra></extra>"
)
fig = go.Figure()
for trace in [trace1, trace2, trace3]:
    fig.add_trace(trace)
fig.update_layout(
    yaxis_tickformat=".1%",
    template="plotly_white",
    showlegend=False
)
fig.show()

# FIGURE 2

In [ ]:
trace1 = ff.create_distplot(
    [daily], 
    group_labels=["actual"], 
    show_rug=False, 
    show_hist=False
)

m = daily.mean()
s = daily.std()
grid = np.linspace(m - 3 * s, m + 3 * s, 201)
trace2 = go.Scatter(
    x=grid, 
    y=norm.pdf(grid, loc=m, scale=s), 
    mode="lines", 
    name="normal"
)

fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.update_layout(
    yaxis_tickvals=[],
    xaxis_tickformat=".0%",
    title=dict(
        text=name + " Daily Returns",
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    ),
    template="plotly_white",
)
fig.show()


# FIGURE 3

In [ ]:
trace1 = ff.create_distplot(
    [monthly], 
    group_labels=["actual"], 
    show_rug=False, 
    show_hist=False
)

m = monthly.mean()
s = monthly.std()
grid = np.linspace(m - 3 * s, m + 3 * s, 201)
trace2 = go.Scatter(
    x=grid, 
    y=norm.pdf(grid, loc=m, scale=s), 
    mode="lines", 
    name="normal"
)

fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.update_layout(
    yaxis_tickvals=[],
    xaxis_tickformat=".0%",
    title=dict(
        text=name + " Monthly Returns",
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    ),
    template="plotly_white",
)
fig.show()


# FIGURE 4

In [ ]:
trace1 = ff.create_distplot(
    [annual], 
    group_labels=["actual"], 
    show_rug=False, 
    show_hist=False
)

m = annual.mean()
s = annual.std()
grid = np.linspace(m - 3 * s, m + 3 * s, 201)
trace2 = go.Scatter(
    x=grid, 
    y=norm.pdf(grid, loc=m, scale=s), 
    mode="lines", 
    name="normal"
)

fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.update_layout(
    yaxis_tickvals=[],
    xaxis_tickformat=".0%",
    title=dict(
        text=name + " Annual Returns",
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    ),
    template="plotly_white",
)
fig.show()

